#### This notebook is to clean data and label the derived variables
1. remove missing values of the key columns
2. remove placeholder values such as tbd from the key columns
3. remove duplicates of the key columns
4. identify the variables if they are derived

export data sets as clean data sets

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',500)  

In [3]:
# Load datasets
file_path_input = "data/raw/ABS-MOPS Variables - December 11 2024.xlsm"
file_path_mdr = "data/raw/mdr Variables 1.xlsx"

input_df = pd.read_excel(file_path_input, sheet_name="Data Sheet", header=12).rename(columns={'Unnamed: 3':'Legacy Variable'})
mdr_df = pd.read_excel(file_path_mdr)

input_columns_to_check = ["Legacy Variable", "Variable Name *", "Description *"]
mdr_columns_to_check = ["name", "definition"]

C:\Users\cyy12\mdr\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
# input_df[input_columns_to_check].head()

In [5]:
# mdr_df[mdr_columns_to_check].head()

In [6]:
# Remove rows with any null values and remove duplicates
print(input_df.shape)
print(mdr_df.shape)
input_df = input_df.dropna(subset=input_columns_to_check)
mdr_df = mdr_df.dropna(subset=mdr_columns_to_check)
print(input_df.shape)
print(mdr_df.shape)
input_df = input_df.drop_duplicates(subset=["Variable Name *"])
mdr_df = mdr_df.drop_duplicates(subset=["name"])
print(input_df.shape)
print(mdr_df.shape)

(395, 28)
(41626, 9)
(392, 28)
(41451, 9)
(371, 28)
(37836, 9)


In [9]:
# Remove place holders like TBD

placeholder_vars = ['tbd'] # lower case

for var in placeholder_vars:
    for col in mdr_columns_to_check:
        to_drop = mdr_df[mdr_df[col].str.lower()==var].index
        mdr_df = mdr_df.drop(to_drop)

for var in placeholder_vars:
    for col in input_columns_to_check:
        to_drop = input_df[input_df[col].str.lower()==var].index
        input_df = input_df.drop(to_drop)

In [10]:
# double check null 
missing_columns = input_df[input_columns_to_check].isnull().any()
print('if still missing values in input df:\n',missing_columns)

missing_columns = mdr_df[mdr_columns_to_check].isnull().any()
print('if still missing values in mdr df:\n',missing_columns)

if still missing values in input df:
 Legacy Variable    False
Variable Name *    False
Description *      False
dtype: bool
if still missing values in mdr df:
 name          False
definition    False
dtype: bool


In [11]:
# double check placeholders:
for var in placeholder_vars:
    print(f'placeholder value: {var}')
    
    df_i = input_df[input_columns_to_check].map(lambda x: x.lower() if isinstance(x, str) else x)
    value_exists = (df_i == var).any().any()
    print(f'if any {var} in input df: \n',value_exists)
    
    df_m = mdr_df[mdr_columns_to_check].map(lambda x: x.lower() if isinstance(x, str) else x)
    value_exists = (df_m == var).any().any()
    print(f'if any {var} in mdr df: \n',value_exists)

placeholder value: tbd
if any tbd in input df: 
 False
if any tbd in mdr df: 
 False


In [19]:
print(input_df.shape)
print(mdr_df.shape)

(371, 28)
(36618, 9)


### find the variables that are derived. --- per client instructions:
* For the MDR names, there is a 'D' in front of the data type abbreviation. For instance, VALUE_ADDED_DVAL is the derived dollar value of the 'value added' variable concept. If this variable were not derived it would be named VALUE_ADDED_VAL
* does the description/definition of the variable include the word "derived"? 
* is there a non-null rules formula value for the variable. If so, this variable is derived. The rules formula column exists for all AIES variables but may not exist for the other variables. You would have to pull it from the aies_variable_view in mdr prod. 

EXCLUDE the derived variables in the MDR file before attempting to run the model and make matches. 

In [22]:
# #  manually check if variables are derived
# mdr_df['derived'] = 'no'
# mask_derived = (mdr_df['definition'].str.lower().str.contains('derive')) | (mdr_df['name'].str.contains('_D')) \
#                | (mdr_df['name'].str.lower().str.contains('derive'))

# ## ? not null rules .... 

# mdr_df.loc[mask_derived, 'derived'] = 'yes'

# # to take a look at if anything else look like the derived in names
# mdr_df[mdr_df['derived'] == 'yes'][['name','definition','derived']].to_excel('data/dbl_chk/mdr_derived_variables_check.xlsx', index=False)

In [28]:
# define derived
derived_data_type_list = ['_DVAL', '_DSUM']

mdr_df['derived'] = 'no'
mask_derived = (mdr_df['name'].str.contains('|'.join(derived_data_type_list), case=False, na=False))
mdr_df.loc[mask_derived, 'derived'] = 'yes'
mdr_df = mdr_df[mdr_df['derived'] == 'no']

input_df['derived'] = 'no'
mask_derived = (input_df['Legacy Variable'].str.contains('|'.join(derived_data_type_list), case=False, na=False)) | (input_df['Variable Name *'].str.contains('|'.join(derived_data_type_list), case=False, na=False))
input_df.loc[mask_derived, 'derived'] = 'yes'
input_df = input_df[input_df['derived'] == 'no']

In [30]:
print(input_df.shape)
print(mdr_df.shape)

(355, 29)
(36524, 10)


In [30]:
# Save cleaned data to new files
input_df.to_excel("data/cleanInput.xlsx", index=False)
mdr_df.to_excel("data/cleanMDR.xlsx", index=False)